In [138]:
%run setup.py
%reload_ext autoreload 
%autoreload 2

from ipyleaflet import *
from ipywidgets import Text, HTML, Tab
import ipywidgets

import cba as cba
from cba_result import CbaResult
from section import Section

<Figure size 432x288 with 0 Axes>

In [189]:
def generate_outputs(df : pd.DataFrame, drop_detail_cols : Bool, keep_cols=[]):
    cba_model = cba.CostBenefitAnalysisModel()
    
    inputs = df.apply(lambda row: Section.from_row(row), axis=1)
    output = [cba_model.compute_cba_for_section(i) for i in inputs]
    output = pd.DataFrame([e.to_dict() for e in output])
    inputs = pd.DataFrame([i.to_dict() for i in inputs])
    
    if drop_detail_cols:
        cols_to_drop = [e for e in list(output.columns) if re.match('.*_[0-9].*', e) and e not in keep_cols]
        # print(f"dropping: {cols_to_drop}")
        output = output.drop(columns=cols_to_drop)
    
    map_data = df.merge(output, left_on='way_id_district', right_on='orma_way_id')
    map_data = map_data.merge(inputs[['orma_way_id', 'roughness', 'road_length']], left_on='way_id_district', right_on='orma_way_id')
    
    map_data.sort_values(by=['work_year', 'npv_cost'], ascending=[True, False], inplace=True)
    map_data['cum_investment'] = np.cumsum(map_data['work_cost'])
    map_data['cum_npv'] = np.cumsum(map_data.npv)

    return map_data

In [209]:
gdf = gpd.read_file(abspath(join('..', 'data', 'sections_generated.gpkg')))

keep_cols = [f'iri_{x}_{y}' for x in ['projection', 'base'] for y in [1,3,5,8,10]] # ['iri_projection_1', 'iri_projection_5', 'iri_projection_8', 'iri_projection_10', 'iri_base_1', 'iri_base_5', 'iri_base_8', 'iri_base_10']

map_gdf = generate_outputs(gdf, True, keep_cols=keep_cols)
# keep_cols

In [294]:
box = ipywidgets.VBox()
box.children = [generate_map(map_gdf), add_tabs()]
box

HTML(value='<style>.budget_label { font-size: 8pt; color: #BBB; }</style>')

HTML(value='<style>.budget_total { font-size: 20pt; width: 100%; text-align: center}</style>')

In [291]:
x=111.213213
f"{x:.2f}"

'111.21'

In [308]:
# def update_sa3_box(feature,  **kwargs):
#     sa3_ = feature['properties']['SA3_NAME16']
#     trips_for_sa3 = survey_by_sa3_and_mode.loc[survey_by_sa3_and_mode.SA3_NAME16 == sa3_]
#     total_trips = trips_for_sa3.trips.sum()
    
#     html_sa3.value = '''
#         <h3><b>{}</b></h3>
#         <h4>SA4 {}</h4>
#         <h4>trips: {:.2f}</h4>
#     '''.format(feature['properties']['SA3_NAME16'],
#                feature['properties']['SA4_NAME16'],
#                total_trips)
# html_sa3 = HTML('''Hover over a SA''')
# html_sa3.layout.margin = '0px 10px 0px 10px'
# control_sa3 = WidgetControl(widget=html_sa3, position='topright')
# m.add_control(control_sa3)

# sa3_data.on_hover(update_sa3_box)

def generate_map(map_gdf):
    
    center = (22.770006, 104.984655)
    m = Map(center=center, zoom=10, basemap=basemaps.CartoDB.Positron)
    m.layout.height="800px"

    attr = ipywidgets.HTML("Hover on a link", layout=ipywidgets.Layout(width='30%'))
    add_roads(m, map_gdf, attr)
    add_budget(m, map_gdf)
    add_dropdown(m)
    

    return ipywidgets.HBox([m, attr])
generate_map(map_gdf.head())

HTML(value='<style>.budget_label { font-size: 8pt; color: #BBB; }</style>')

HTML(value='<style>.budget_total { font-size: 20pt; width: 100%; text-align: center}</style>')

In [277]:
%matplotlib inline
def add_tabs():
    
    tab = Tab()
    tab.children = [generate_cumulative_npv(map_gdf), 
                    generate_network_stats(map_gdf), 
                    generate_priorities(map_gdf)]
    tab.set_title(index=0, title="Cumulative NPV")
    tab.set_title(index=1, title='Network Stats')
    tab.set_title(index=2, title='Budget Effects')
    
    
    return tab

In [276]:
def generate_priorities(gdf):
    canvas = ipywidgets.Output()
    def update_priorities(change):
        with canvas: 
            canvas.clear_output()
            fig, ax = plt.subplots(constrained_layout=True, figsize=(15,5))
            fig.canvas.toolbar_position = 'bottom'
            # ax.grid(True)
            
            years = [1,3,5,8]
            total_budget = sum(gdf.work_cost)
            total_length = sum(gdf.road_length)
            
            base_iri = [sum(gdf[f'iri_base_{x}'] * map_gdf['road_length'])/total_length for x in years]
            def get_threshold_iri(perc):
                threshold = perc * total_budget
                do_df, dont_df = gdf.query('cum_investment < @threshold'), gdf.query('cum_investment >= @threshold')
                                 
                do_numbers = [sum(do_df[f'iri_projection_{x}'] * do_df['road_length']) for x in [1,3,5,8]]
                dont_numbers = [sum(dont_df[f'iri_base_{x}'] * dont_df['road_length']) for x in [1,3,5,8]]
                return [(x+y) / total_length for x,y in zip(do_numbers,dont_numbers)]
            thresholds = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
            data = {f'{p*100}%' : get_threshold_iri(p) for p in thresholds}
            data['year'] = years = [y + 2021 for y in years]
            df = pd.DataFrame(data=data).set_index('year')
            # display(df)
            sns.lineplot(data=df, ax=ax, palette=sns.color_palette("light:#5A9",6))
            c = sns.color_palette("husl", 9)[7]
            
            sns.lineplot(x=years, y=get_threshold_iri(change['new']/100.0), ax=ax, color=c, 
                         label=f"Custom: {change['new']}%")
            
            ax.set_title("Roughness Progression By Percentage Investment", fontsize=18)
            
            plt.show()
            return None
    # update_cumulative_npv(gdf)
    slider = ipywidgets.FloatSlider(value=90.0, min=0, max=100.0, description='Budget %:', continuous_update=False)
    slider.observe(update_priorities, names='value')
    update_priorities({'new': 90.0})
    
    return ipywidgets.VBox([slider, canvas])


In [160]:
def generate_cumulative_npv(gdf):
    canvas = ipywidgets.Output()
    def update_cumulative_npv(gdf):
        
        with canvas: 
            canvas.clear_output()
            fig, ax = plt.subplots(constrained_layout=True, figsize=(15,5))
            fig.canvas.toolbar_position = 'bottom'
            ax.grid(True)

            sns.scatterplot(data=gdf, ax=ax, x='cum_investment', y='cum_npv', hue='work_year', palette='husl')
            ax.set_xlabel("Investment $M", fontsize=22)
            ax.set_ylabel("NPV $M", fontsize=2)
            ax.set_title(f'NPV vs Investment', fontsize=2)
            plt.show()
            return None
    update_cumulative_npv(gdf)
    
    return canvas

In [151]:
def generate_network_stats(gdf):
    canvas = ipywidgets.Output()
    def update_network_stats(gdf):
        
        with canvas: 
            canvas.clear_output()
            df = gdf.loc[:,['roughness', 'road_length']]
            def categorise_roughness(r):
                if r < 2.5: return "<2.5"
                elif r < 4: return "2.5-4.0"
                elif r < 6: return "4.0-6.0"
                elif r < 10: return "6.0-10.0"
                elif r < 14: return "10.0-14.0"
                else : return "14.0+"
            df["roughness_label"] = df['roughness'].apply(categorise_roughness)

            fig = plt.figure(constrained_layout=True, figsize=(15,5))
            ax = plt.subplot(1,2,1)
            ax.grid(True)
            
            # sns.barplot(data=df, ax=ax, x='roughness_label', y='road_length')
            sns.histplot(data=df, x='roughness', ax=ax, stat='density', weights='road_length', 
                         binrange=(0,16), bins=16)
            ax.set_xlabel("Roughness", fontsize=16)
            ax.set_ylabel("% of Network", fontsize=16)
            
            #Seaborn Horizontal barplot
            df = df.groupby('roughness_label').sum()[['road_length']].reset_index()
            ax = plt.subplot(1,2,2)
            plt.pie(x=df['road_length'], autopct="%.1f%%", 
                    labels=df['roughness_label'], pctdistance=0.5, colors=sns.color_palette("Set2", 9))
            ax.set_title("Roughness Distribution", fontsize=16)
            
            plt.show()
            return None
    update_network_stats(gdf)
    
    return canvas



In [157]:
def add_dropdown(m):
    global z_mode
    z_mode = "Work Year"

    def on_click_dropdown_options(change):
        global z_mode, z_time
        z_mode = change['new']
        # update_zones_box_comp()

    dropdown_options = ipywidgets.Dropdown(
        options=['Work Year', 'Relative CBR', 'Province'],
        value=z_mode,
        description='Color By'
    )

    dropdown_options.observe(on_click_dropdown_options, 'value')
    widget_option = WidgetControl(widget=dropdown_options, position='topright')

    m.add_control(widget_option)

In [324]:
def create_li(desc, value):
    return f'''
        <li>
          <div class='attr_label'>{desc}:</div>
          <div class='attr_value'>{value}</div>
        </li>
    '''

#MM_ID': '032HG00001', 'aadt': None, 'condition': '3', 'cum_investment': 595.9695185875752, 'cum_npv': 187.8679445588673, 'district': 'Huyện Hoàng Su Phì', 'eirr': 0.20888198812785586, 'esa_loading': 0.0030787749999999997, 'iri': None, 'iri_base_1': 6, 'iri_base_10': 2, 'iri_base_3': 6.303749999999999, 'iri_base_5': 6.622877343749998, 'iri_base_8': 7.132114522009274, 'iri_projection_1': 6, 'iri_projection_10': 3.863345117187498, 'iri_projection_3': 6.303749999999999, 'iri_projection_5': 6.622877343749998, 'iri_projection_8': 3.677187499999999, 'length': 55.7246586607537, 'link_class': '5', 'management': '2', 'name': 'Bắc Quang-Xín Mần', 'npv': 10.045667331462528, 'npv_cost': 0.1704711924937048, 'npv_km': 0.1802732860620928, 'orma_way_id_x': '614835_302', 'orma_way_id_y': '614835_302', 'province': 'Hà Giang', 'road end location': 'thị trấn Cốc Pài, huyện Xín Mần', 'road number': 'ĐT.177', 'road start location': ' Km245, QL.2 (thị trấn Tân Quang, huyện Bắc Quang)', 'road_length': 55.7246586607537, 'roughness': 6, 'section_articulated_truck': None, 'section_commune_gso': None, 'section_delivery_vehicle': None, 'section_four_wheel': None, '
    
things_to_show = [
    ('ORMA ID', 'orma_way_id_x'),
    ('Name', 'name'),
    ('District', 'district'),
    ('Surface', 'surface'),
    
    ('Work Name', 'work_name'),
    ('Work Class', 'work_class'),
    ('Work Year', 'work_year'),
    ('Work Cost', 'work_cost'),
    
]
def add_roads(m, df, attr):
    map_geo =  GeoData(geo_dataframe = map_gdf,
                   style={'color': 'purple', 'opacity':3, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Roads')

    layer = m.add_layer(map_geo)
    
    def update_attributes(feature,  **kwargs):
        
        #print(feature['properties'].keys())
        inner_str="\n".join([create_li(desc, feature['properties'][key]) for desc,key in things_to_show])
        attr.value=f"<ul>{inner_str}</ul>"#  + str(feature['properties'].keys())
    map_geo.on_hover(update_attributes)
    
generate_map(map_gdf.head())
#map_gdf.head()

HTML(value='<style>.budget_label { font-size: 8pt; color: #BBB; }</style>')

HTML(value='<style>.budget_total { font-size: 20pt; width: 100%; text-align: center}</style>')

In [292]:
def add_budget(m, gdf):
    total_budget = sum(gdf['work_cost'])
    budget_callout = HTML(f'''
        <div class='budget_label'>Total Budget ($USD)</div>
        <div class='budget_total'>${total_budget:.2f}M </div>
    ''')
    budget_callout.layout.margin = '5px 10px 5px 10px'
    budget_control = WidgetControl(widget=budget_callout, position='bottomleft')

    m.clear_controls()
    m.add_control(budget_control)
    display(HTML("<style>.budget_label { font-size: 8pt; color: #BBB; }</style>"))
    display(HTML("<style>.budget_total { font-size: 20pt; width: 100%; text-align: center}</style>"))


In [7]:
survey_by_sa3_and_mode = survey_data.groupby(['SA3_NAME16', 'estimation_mode']).weight.sum().to_frame().reset_index().rename(columns={'weight':'trips'})

In [8]:
# html_sa3 = HTML('''Hover over a SA''')
# html_sa3.layout.margin = '0px 10px 0px 10px'
# control_sa3 = WidgetControl(widget=html_sa3, position='topright')
# m.add_control(control_sa3)

# def update_sa3_box(feature,  **kwargs):
#     sa3_ = feature['properties']['SA3_NAME16']
#     trips_for_sa3 = survey_by_sa3_and_mode.loc[survey_by_sa3_and_mode.SA3_NAME16 == sa3_]
#     total_trips = trips_for_sa3.trips.sum()
    
#     html_sa3.value = '''
#         <h3><b>{}</b></h3>
#         <h4>SA4 {}</h4>
#         <h4>trips: {:.2f}</h4>
#     '''.format(feature['properties']['SA3_NAME16'],
#                feature['properties']['SA4_NAME16'],
#                total_trips)

# sa3_data.on_hover(update_sa3_box)

In [9]:
html_sa3_1 = HTML('''Click on SA3 for trips by mode''')
html_sa3_1.layout.margin = '0px 10px 0px 10px'
control_sa3_1 = WidgetControl(widget=html_sa3_1, position='bottomleft')
m.add_control(control_sa3_1)

def update_sa3_box_1(feature,  **kwargs):
    sa3_ = feature['properties']['SA3_NAME16']
    trips_for_sa3 = survey_by_sa3_and_mode.loc[survey_by_sa3_and_mode.SA3_NAME16 == sa3_]
    total_trips = trips_for_sa3.trips.sum()
    
    #mode_string = trips_for_sa3.drop(columns=['SA3_NAME16']).style.hide_index().render()
    mode_string = "<h4>{}: {:.2f}</h4>".format(sa3_, total_trips)
    for mode in trips_for_sa3.estimation_mode.values:
        trips = trips_for_sa3.loc[trips_for_sa3.estimation_mode == mode].trips.values[0]
        mode_string += "<h5>{}: {:.2f}%</h5>".format(mode, 100.0 * trips / total_trips)
    
    html_sa3_1.value = mode_string

sa3_data.on_click(update_sa3_box_1)